In [2]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D



In [23]:
import collections
import os
import cv2
import pandas as pd
import numpy as np

In [5]:
DATA_FOLDER = 'train'
file_names = []
for root, dirs, files_list in os.walk(DATA_FOLDER):
    file_names += files_list
print('Total number of train files:  ', len(file_names))
data = []
for file in file_names:
    file_path = os.path.join(DATA_FOLDER, file)
    data.append(cv2.imread(file_path))
data = np.array(data)
print('Samples: ', data.shape[0],
      'x-axis width: ', data.shape[1],
      'y-axis height: ', data.shape[2],
      'colors: ', data.shape[3])

Total number of train files:   48896


In [ ]:
# tenho que ler aqui.

In [13]:
labels = pd.read_csv('train.truth.csv', sep=',')

In [19]:
DATA_FOLDER = 'train'
data = []
for file in labels.fn.tolist():
    file_path = os.path.join(DATA_FOLDER, file)
    data.append(cv2.imread(file_path))
data = np.array(data)
print('Samples: ', data.shape[0],
      'x-axis width: ', data.shape[1],
      'y-axis height: ', data.shape[2],
      'colors: ', data.shape[3])

In [40]:
NOT_ROTATE = -1
def rot_corrections(label_name):
    if label_name == 'rotated_left':
        return cv2.ROTATE_90_CLOCKWISE
    elif label_name == 'upright':
        return NOT_ROTATE
    elif label_name == 'rotated_right':
        return cv2.ROTATE_90_COUNTERCLOCKWISE
    elif label_name == 'upside_down':    
        return cv2.ROTATE_180

def correct_image(image, rot_correction):
    if rot_correction == NOT_ROTATE:
        return image
    else:
        return cv2.rotate(image, rot_correction)    
    

In [37]:
labels['rot_corrections'] = labels.label.apply(rot_corrections)

In [42]:
for i in range(15):
    rotated = correct_image(data[i], labels.loc[i, 'rot_corrections'])
    cv2.imwrite(str(i) + 'image.jpg', data[i])
    cv2.imwrite(str(i) + 'rotated.jpg', rotated)

In [ ]:
cv2.imwrite('teste-90clock.jpg', img_rotate_90_clockwise)
cv2.imwrite('teste-0.jpg', data[0])

In [22]:
img_rotate_90_clockwise = cv2.rotate(data[0], cv2.ROTATE_90_CLOCKWISE)
cv2.imwrite('teste-90clock.jpg', img_rotate_90_clockwise)
cv2.imwrite('teste-0.jpg', data[0])

True

In [28]:
i = 48877
img_rotate_90_clockwise = cv2.rotate(data[i], cv2.ROTATE_90_CLOCKWISE)
cv2.imwrite(str(i) + 'teste-90clock.jpg', img_rotate_90_clockwise)
cv2.imwrite(str(i) + 'teste.jpg', data[i])

True

In [29]:
cv2.ROTATE_90_CLOCKWISE

0

In [ ]:
cv2.ROTATE_90_CLOCKWISE
cv2.ROTATE_90_COUNTERCLOCKWISE
cv2.ROTATE_180

In [31]:
cv2.ROTATE_90_COUNTERCLOCKWISE

2

In [25]:
labels[labels.label == 'upright']

,fn,label
2,0-10126400_1964-07-07_2010.jpg,upright
9,0-10416800_1907-01-08_1967.jpg,upright
15,0-10898800_1951-06-03_1950.jpg,upright
23,0-11435300_1959-12-26_1950.jpg,upright
25,0-1157400_1938-10-01_1961.jpg,upright
...,...,...
48877,89-9041089_1953-08-01_1978.jpg,upright
48878,89-9209089_1986-04-28_2014.jpg,upright
48882,89-9332189_1974-07-23_2014.jpg,upright
48889,89-9641289_1900-12-21_1960.jpg,upright


In [24]:
collections.Counter(labels.label)

Counter({'rotated_left': 12266,
         'upright': 12212,
         'rotated_right': 12314,
         'upside_down': 12104})

In [14]:
labels.head(3)

,fn,label
0,0-10049200_1891-09-16_1958.jpg,rotated_left
1,0-10110600_1985-09-17_2012.jpg,rotated_left
2,0-10126400_1964-07-07_2010.jpg,upright


In [ ]:
batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models_2')
model_name = 'keras_cifar10_trained_model.h5'

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255